# simple nn model fitting

Notebook will run a very simple nn on the training points

In [1]:
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
# import librosa
# import librosa.display
# import sklearn as skl
# import sklearn.utils
# import IPython.display as ipd
# from sklearn.model_selection import train_test_split
# import ast

from sklearn.preprocessing import OneHotEncoder

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
#define network class
#used some help from https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

class Net(nn.Module):
    def __init__(self, input_size, output_size = 1):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(input_size, int(input_size/2))
        self.layer2 = nn.Linear(int(input_size/2), int(input_size/4))
        self.layer3 = nn.Linear(int(input_size/4), output_size)

    # x represents our data
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

        

In [45]:
#import training data
train_data = torch.tensor(np.load('data/train_data.npy').T).float()[:1000]
labels = torch.tensor(np.load('data/train_labels.npy')).float()
n_dim, n_training_points = train_data.shape
print("n_dim = ", n_dim)
print('n_training_points = ', n_training_points )

#get testing data
test_data = torch.tensor(np.load('data/test_data.npy').T).float()[:1000]
test_labels = torch.tensor(np.load('data/test_labels.npy')).float()
_, n_testing_points = test_data.shape
print("n_dim = ", n_dim)
print('n_training_points = ', n_testing_points )
print(test_data)



enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(torch.cat((labels, test_labels)).unsqueeze(1)) #takes a 2d array as input and labels is 1d; need labels for all training and testing points
print(len(enc.categories_))
labels = enc.transform(labels.unsqueeze(1)).toarray()
labels = torch.tensor(labels).float()
print(labels.shape)
_, n_categories = labels.shape #number of categories is out output size for our network


#encode testing labels
test_labels = enc.transform(test_labels.unsqueeze(1)).toarray()
test_labels = torch.tensor(test_labels).float()
print(test_labels.shape)


n_dim =  1000
n_training_points =  50
n_dim =  1000
n_training_points =  25
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.0077, -0.1812, -0.1608,  ...,  0.2518,  0.1378,  0.1512],
        [-0.0131, -0.1836, -0.1614,  ...,  0.2579,  0.1327,  0.1578],
        [-0.0178, -0.1850, -0.1577,  ...,  0.2488,  0.1278,  0.1573]])
1
torch.Size([50, 18])
torch.Size([25, 18])


In [46]:
model = Net(n_dim, n_categories)
print(model)
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.MSELoss()

Net(
  (layer1): Linear(in_features=1000, out_features=500, bias=True)
  (layer2): Linear(in_features=500, out_features=250, bias=True)
  (layer3): Linear(in_features=250, out_features=18, bias=True)
)


In [47]:
max_epochs = 200
for epoch in range(max_epochs):
    optimizer.zero_grad()   # zero the gradient buffers
#     print(train_data.shape)
    output = model(train_data.T)
#     print(output.shape)
#     print(labels.shape)
#     print(type(output))
#     print(type(labels))
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()    # Does the update
    print("loss = ", loss)

loss =  tensor(0.0592, grad_fn=<MseLossBackward>)
loss =  tensor(0.0546, grad_fn=<MseLossBackward>)
loss =  tensor(0.0508, grad_fn=<MseLossBackward>)
loss =  tensor(0.0476, grad_fn=<MseLossBackward>)
loss =  tensor(0.0451, grad_fn=<MseLossBackward>)
loss =  tensor(0.0426, grad_fn=<MseLossBackward>)
loss =  tensor(0.0401, grad_fn=<MseLossBackward>)
loss =  tensor(0.0381, grad_fn=<MseLossBackward>)
loss =  tensor(0.0364, grad_fn=<MseLossBackward>)
loss =  tensor(0.0349, grad_fn=<MseLossBackward>)
loss =  tensor(0.0336, grad_fn=<MseLossBackward>)
loss =  tensor(0.0324, grad_fn=<MseLossBackward>)
loss =  tensor(0.0312, grad_fn=<MseLossBackward>)
loss =  tensor(0.0300, grad_fn=<MseLossBackward>)
loss =  tensor(0.0289, grad_fn=<MseLossBackward>)
loss =  tensor(0.0278, grad_fn=<MseLossBackward>)
loss =  tensor(0.0270, grad_fn=<MseLossBackward>)
loss =  tensor(0.0262, grad_fn=<MseLossBackward>)
loss =  tensor(0.0257, grad_fn=<MseLossBackward>)
loss =  tensor(0.0252, grad_fn=<MseLossBackward>)


loss =  tensor(0.0060, grad_fn=<MseLossBackward>)
loss =  tensor(0.0059, grad_fn=<MseLossBackward>)
loss =  tensor(0.0057, grad_fn=<MseLossBackward>)
loss =  tensor(0.0058, grad_fn=<MseLossBackward>)
loss =  tensor(0.0058, grad_fn=<MseLossBackward>)
loss =  tensor(0.0056, grad_fn=<MseLossBackward>)
loss =  tensor(0.0056, grad_fn=<MseLossBackward>)
loss =  tensor(0.0056, grad_fn=<MseLossBackward>)
loss =  tensor(0.0056, grad_fn=<MseLossBackward>)
loss =  tensor(0.0055, grad_fn=<MseLossBackward>)
loss =  tensor(0.0055, grad_fn=<MseLossBackward>)
loss =  tensor(0.0055, grad_fn=<MseLossBackward>)
loss =  tensor(0.0054, grad_fn=<MseLossBackward>)
loss =  tensor(0.0054, grad_fn=<MseLossBackward>)
loss =  tensor(0.0054, grad_fn=<MseLossBackward>)
loss =  tensor(0.0053, grad_fn=<MseLossBackward>)
loss =  tensor(0.0053, grad_fn=<MseLossBackward>)
loss =  tensor(0.0052, grad_fn=<MseLossBackward>)
loss =  tensor(0.0052, grad_fn=<MseLossBackward>)
loss =  tensor(0.0052, grad_fn=<MseLossBackward>)


In [48]:
print(output.shape)

torch.Size([50, 18])


In [49]:
count = 0
#count number predicted correctly
for i in range(len(output)):
    idx_output = torch.argmax(output[i])
    idx_truth = torch.argmax(labels[i])
    if idx_output == idx_truth:
        count += 1
    else:
        pass #do nothing if not equal
print("percent predicted correctly is ", 100*count/len(output), '%')
            

percent predicted correctly is  96.0 %


In [51]:
count = 0
test_output = model(test_data.T)
#count number predicted correctly
for i in range(len(test_output)):
    idx_output = torch.argmax(test_output[i])
    idx_truth = torch.argmax(test_labels[i])
    print("idx_output = ", idx_output, ": idx_truth = ", idx_truth)
    if idx_output == idx_truth:
        count += 1
    else:
        pass #do nothing if not equal
print("percent predicted correctly is ", 100*count/len(output), '%')

idx_output =  tensor(3) : idx_truth =  tensor(4)
idx_output =  tensor(12) : idx_truth =  tensor(2)
idx_output =  tensor(3) : idx_truth =  tensor(2)
idx_output =  tensor(3) : idx_truth =  tensor(3)
idx_output =  tensor(4) : idx_truth =  tensor(1)
idx_output =  tensor(5) : idx_truth =  tensor(14)
idx_output =  tensor(5) : idx_truth =  tensor(14)
idx_output =  tensor(5) : idx_truth =  tensor(10)
idx_output =  tensor(3) : idx_truth =  tensor(3)
idx_output =  tensor(5) : idx_truth =  tensor(10)
idx_output =  tensor(5) : idx_truth =  tensor(10)
idx_output =  tensor(5) : idx_truth =  tensor(10)
idx_output =  tensor(2) : idx_truth =  tensor(15)
idx_output =  tensor(5) : idx_truth =  tensor(15)
idx_output =  tensor(5) : idx_truth =  tensor(6)
idx_output =  tensor(5) : idx_truth =  tensor(6)
idx_output =  tensor(3) : idx_truth =  tensor(6)
idx_output =  tensor(2) : idx_truth =  tensor(6)
idx_output =  tensor(4) : idx_truth =  tensor(1)
idx_output =  tensor(3) : idx_truth =  tensor(0)
idx_output 